In [ ]:
# change color of ann image if not black color in the image change to white in folder FoodSeg103/Images/ann_dir/{train/test} and 
# save in folder FoodSeg103/Images/ann_dir/{train/test}_new

import os
import cv2
import numpy as np
from PIL import Image

# path to ann_dir
path = 'FoodSeg103/Images/ann_dir/train'
path_new = 'FoodSeg103/Images/ann_dir/train_new'

# Create folder if not exist
if not os.path.exists(path_new):
    os.makedirs(path_new)

# get all file name in ann_dir
files = os.listdir(path)
 
# loop through all files
for file in files:
    # read image
    img = cv2.imread(os.path.join(path, file))
    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # get all pixel value
    pixels = np.unique(gray)
    # if pixel value is not 0 (black) change to 255 (white)
    if len(pixels) > 1:
        img[gray != 0] = 255
    # save image
    cv2.imwrite(os.path.join(path_new, file), img)
    
# path to ann_dir
path = 'FoodSeg103/Images/ann_dir/test'
path_new = 'FoodSeg103/Images/ann_dir/test_new'

# Create folder if not exist
if not os.path.exists(path_new):
    os.makedirs(path_new)

# get all file name in ann_dir
files = os.listdir(path)

# loop through all files
for file in files:
    # read image
    img = cv2.imread(os.path.join(path, file))
    # convert to gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # get all pixel value
    pixels = np.unique(gray)
    # if pixel value is not 0 (black) change to 255 (white)
    if len(pixels) > 1:
        img[gray != 0] = 255
    # save image
    cv2.imwrite(os.path.join(path_new, file), img)
    


In [26]:
# resize all image in 
# FoodSeg103/Images/ann_dir/{test_new/train_new} and FoodSeg103/Images/img_dir/{test/train}
# to 400x400 and save in folder FoodSeg103_new/Images/{ann_dir/img_dir}/{test/train}
# Path: resize_image.ipynb
import os
import cv2

def resize_images(source_path, target_path):
    # Create target folder if it doesn't exist
    if not os.path.exists(target_path):
        os.makedirs(target_path)

    # Get all file names in the source folder
    files = os.listdir(source_path)

    # Loop through all files
    for file in files:
        # Read image
        img = cv2.imread(os.path.join(source_path, file))
        # Resize image
        img = cv2.resize(img, (400, 400))
        # Save image
        cv2.imwrite(os.path.join(target_path, file), img)

# Define paths for ann_dir and img_dir
directories = [
    ('FoodSeg103/Images/ann_dir/train_new', 'FoodSeg103_new/Images/ann_dir/train'),
    ('FoodSeg103/Images/ann_dir/test_new', 'FoodSeg103_new/Images/ann_dir/test'),
    ('FoodSeg103/Images/img_dir/train', 'FoodSeg103_new/Images/img_dir/train'),
    ('FoodSeg103/Images/img_dir/test', 'FoodSeg103_new/Images/img_dir/test')
]

# Loop through the directories
for source_path, target_path in directories:
    resize_images(source_path, target_path)


In [42]:
# change strucutre of dataset from FoodSeg103_new to this structure
# datasets/preprocess/{train/test/val}/images on val is split from test folder

# Path: create_dataset.ipynb
import os
import shutil
import numpy as np

# Define paths for ann_dir and img_dir
directories = [
    ("FoodSeg103_new/Images/ann_dir/train/", "datasets/preprocess/train/ann/"),
    ("FoodSeg103_new/Images/ann_dir/test/", "datasets/preprocess/test/ann/"),
    ("FoodSeg103_new/Images/img_dir/train/", "datasets/preprocess/train/img/"),
    ("FoodSeg103_new/Images/img_dir/test/", "datasets/preprocess/test/img/"),
]

# Loop through the directories
for source_path, target_path in directories:
  
    # Create target folder if it doesn't exist
    if not os.path.exists(target_path):
        os.makedirs(target_path)
        
    # Get all file names in the source folder
    files = os.listdir(source_path)
    # Loop through all files
    for file in files:
        # Copy file
        shutil.copy(
            os.path.join(source_path, file),
            os.path.join(target_path, file),
        )

# split test folder to test and val folder with the same file name from img and ann folder
# Path: create_dataset.ipynb

# Define paths for ann_dir and img_dir
test_ann = "datasets/preprocess/test/ann/"
test_img = "datasets/preprocess/test/img/"
val_ann = "datasets/preprocess/val/ann/"
val_img = "datasets/preprocess/val/img/"

val_image_name = []

# random 20% of test folder to val folder with the same file name
for file in os.listdir(test_img):
    if np.random.rand() < 0.2:
        val_image_name.append(file)
        
# Create target folder if it doesn't exist
if not os.path.exists(val_ann):
    os.makedirs(val_ann)
if not os.path.exists(val_img):
    os.makedirs(val_img)
    
# Loop through all files
for file in val_image_name:
    # Copy file from test folder to val folder with own file extension
    shutil.copy(
        os.path.join(test_ann, file.replace(".jpg", ".png")),
        os.path.join(val_ann, file.replace(".jpg", ".png")),
    )
    shutil.copy(
        os.path.join(test_img, file),
        os.path.join(val_img, file),
    )
    
# remove file in test folder
for file in val_image_name:
    os.remove(os.path.join(test_ann, file.replace(".jpg", ".png")))
    os.remove(os.path.join(test_img, file))

In [10]:
import numpy as np
from PIL import Image
from skimage import draw
import random
from pathlib import Path
from rasterio import features

def create_label(image_path, label_path):
    arr = np.asarray(Image.open(image_path))

    cords = list(features.shapes(arr, mask=None))[0][0]['coordinates'][0]
    label_line = '0 ' + ' '.join([f'{int(cord[0])/arr.shape[0]} {int(cord[1])/arr.shape[1]}' for cord in cords])
    label_path.parent.mkdir( parents=True, exist_ok=True )
    with label_path.open('w') as f:
        f.write(label_line)

    
for images_dir_path in [Path(f'datasets/preprocess/{x}/ann') for x in ['train', 'val', 'test']]:
    for img_path in images_dir_path.iterdir():
        label_path = img_path.parent.parent / 'labels' / f'{img_path.stem}.txt'
        label_line = create_label(img_path, label_path)

In [ ]:
# copy all dataset in FoodSeg103/Images/img_dir/{test/train/val} and FoodSeg103/Images/labels to datasets/{test/train/val}/images and datasets/{test/train/val}/labels
import shutil

for dataset in ['train', 'test', 'val']:
    shutil.copytree(f'datasets/preprocess/{dataset}', f'datasets/{dataset}/images')
    shutil.copytree(f'datasets/preprocess/{dataset}/labels', f'datasets/{dataset}/labels')
    
# create datasets/{test/train/val}/images.txt
for dataset in ['train', 'test', 'val']:
    with open(f'datasets/{dataset}/images.txt', 'w') as f:
        for img_path in Path(f'datasets/{dataset}/images').iterdir():
            f.write(f'{img_path}\n')

    
    

In [28]:
# move all labels file in FoodSeg103/Images/labels to datasets/{test/train/val}/labels with the same name in datasets/{test/train/val}/images/{image_name}.jpg

for label_dir_path in [Path(f'FoodSeg103/Images/labels')]:
    for label_path in label_dir_path.iterdir():
        for images_dir_path in [Path(f'datasets/{x}/images') for x in ['train', 'test', 'val']]:
            for img_path in images_dir_path.iterdir():
                if img_path.stem == label_path.stem:
                  # print(str(images_dir_path.parent)+'/labels/'+label_path.name)
                  shutil.move(label_path, str(images_dir_path.parent)+'/labels/'+label_path.name)

In [29]:
yaml_content = f'''
train: train/images
val: val/images
test: test/images

names: ['food']
    '''
    
with Path('data.yaml').open('w') as f:
    f.write(yaml_content)